# Estimate single-cell density at observed time point. 

***Pseudodyanmics+*** leverage a concpet called Physics-Informed Neural Networks (PINN) which can solve the forward and inverse problem of PDE. We applies the continuous PINN which requires:
- the model correctly predict density at observed time points and data points
- the model follows the PDE (physics formular) anywhere (collocation point : unobserved cell state) and anytime (collocation time : unobserved time point)


In this notebook, we will go through several density estimator that could served as the label for solving PINN. <font color='Blue'>
1. Gaussian KDE
2. Mellon (the latest)
3. HBE (scalabl)

</font>

Remember that density estimation in high-dimensional space is not retrival. None of the method above captured the true density as the true manifold of the single-cell data is never known.

# Efficient kernel density estimator for atlas level data

Traditionaly gaussidan KDE scales badly to sample size. For atals with millions of cells, we can use a hash-based estimator for sample-efficiency.   
The package `efficient_kde` provide several estimators for accelaration. see the [original github](https://github.com/talwagner/efficient_kde) for details.   
The manuscript by [Moses Charikar and Paris Siminelakis FOCS 2017](https://arxiv.org/abs/1808.10530)

In [ ]:
from efficient_kde.kde import FastLaplacianKDE, kde

# Hyperparameter for HBE
bandwidth = 1
L = 90

In [ ]:
# cell state coordiate
# scale to 0-1
DM_x = adata.obsm['DM_EigenVectors_multiscaled']

DM_min = DM_x.min(axis=0, keepdims=True)
DM_x_range = (DM_x.max(axis=0, keepdims=True) - DM_min)

# update 
norm_DMx = (DM_x - DM_min) / DM_x_range
adata.obsm['DM_EigenVectors_multiscaled'] = norm_DMx

# calculate
DM_normed = [adata[cbs].obsm['DM_EigenVectors_Scaled'] for cbs in CB_ay]


hbe_u = []

for dm in DM_ay:
    # print(dm.shape)
    # kde_fn = gaussian_kde(dm.T, bw_method='silverman')       # take in [n_dim, n_sample]
    hbe = FastLaplacianKDE(dm, bandwidth=0.05, L=30)
    u_t = np.array([hbe.kde(dm[i,:]) for i in range(dm.shape[0])])
    hbe_u.append(u_t)

hbe_u = np.concatenate(hbe_u, axis=0)
clu7_ad.obs['hbe_u_dm30'] = hbe_u
